In [ ]:
cur_dir = !pwd
if cur_dir[0] != "/content":
  print("getting back to content dir")
  %cd /content/
else:
  print("already in /content")  
!rm -rf 046211-project
!git clone https://github.com/ynahum/046211-project.git
%cd 046211-project

/content
Cloning into '046211-project'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 44 (delta 17), reused 25 (delta 6), pack-reused 4
Unpacking objects: 100% (44/44), done.
/content/046211-project


In [ ]:
!pip install learn2learn

     |████████████████████████████████| 604 kB 15.5 MB/s 
     |████████████████████████████████| 2.9 MB 52.4 MB/s 
     |████████████████████████████████| 135 kB 61.3 MB/s 
     |████████████████████████████████| 96 kB 6.2 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 1.4 MB 65.1 MB/s 
     |████████████████████████████████| 3.6 MB 53.3 MB/s 
  Created wheel for learn2learn: filename=learn2learn-0.1.6-cp37-cp37m-linux_x86_64.whl size=945734 sha256=6311b4e64a9d4e896fb5fbe5fcb9a0986fdd68faf38bc3e7f0029ad5768ebbed
  Stored in directory: /root/.cache/pip/wheels/4f/d0/99/d65b01d3a6bda35325b9b2e964c7c17169951c915484029b74
  Created wheel for qpth: filename=qpth-0.0.15-py3-none-any.whl size=15379 sha256=040ebc83633ef6ee33457e01ce20c20049fde0a659a1a5d068a8f23f830fa319
  Stored in directory: /root/.cache/pip/wheels/f6/bb/0f/3af358159c8cfc56654d85ba5069b53ab351dee72f5a57c2ff
  Created wheel for gsutil: filename=gsutil-5.5-py3-none-any.whl 

In [ ]:

from meta_learner_run import run_meta_learner
import copy 
from config import defaults

args = copy.deepcopy(defaults)

run_meta_learner(
    dataset=args['dataset'],
    train_sample_size=args['train_sample_size'],
    n_test_labels=args['n_test_labels'],
    n_shots=args['n_shots'],
    per_task_lr=args['per_task_lr'],
    meta_lr=args['meta_lr'],
    adaptation_steps=args['adaptation_steps'],
    meta_batch_size=args['meta_batch_size'],
    n_epochs=args['n_epochs'])

get tasks
Downloading: /root/data/mini-imagenet-cache-train.pkl
Downloading: /root/data/mini-imagenet-cache-validation.pkl
Downloading: /root/data/mini-imagenet-cache-test.pkl
schedule training
load model (dataset is mini-imagenet)
create meta learner
meta learner train


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


1.4762015342712402 0.380000002682209
1.4229207932949066 0.3700000047683716
1.3721868097782135 0.4450000002980232
1.4929717481136322 0.33500000089406967
1.4870821833610535 0.3750000037252903
